### 📌 **Project Overview**

You're a data scientist at **Nerdwallet,** a mid-sized consumer finance company that aired an advertisement during the **Super Bowl** on **February 10, 2025**. Your goal is to estimate how the ad influenced branded search interest using **causal inference techniques**.

Google Trends data is **normalized** from 0–100, where **100 represents the peak search interest** during the selected time window. This means the data shows *relative* search volume, not absolute counts.

Your goal is to report results using your preferred causal inference model. To do this, 
you'll first explore various potential models, including:

- **Interrupted Time Series (ITS)**
- **Difference-in-Differences (DiD)**
- **Event Studies**
- **Synthetic Control Methods**

You'll want to test the assumptions of these models when relevant. To make your decision on the preferred model and results that you ultimately report, you'll want to consider which models make sense under the circumstances and are supported using historical (pre-event) data. This could include: 

- Running **placebo tests** (to check for false positives)
- Performing **sensitivity analysis** (to test robustness to assumptions)
- Choosing the most credible model and **justifying your findings to stakeholders**

In [3]:
# Import packages as needed for the analysis

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf

from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler



### 📌 **Import data**

You can find the exact time series for both NerdWallet and search interest for relevant business-related topics at this link: https://trends.google.com/trends/explore?date=2024-11-01%202025-02-28&geo=US&q=%2Fm%2F0hzpb7j,credit%20card%20points,travel%20credit%20card,best%20savings%20account&hl=en

NerdWallet is the time series of direct search interest for the company. Search interest for 'credit card points', 'travel credit card', and 'best savings account' help identify the seasonality of interest in companies like NerdWallet. These help control for the seasonality of which users might search for NerdWallet in the absence of the Super Bowl ad. 

Note that when downloading the file, there are some lines of extra space at the top. You can remove these within pd.read_csv(). 


In [ ]:
# Starter code and initial parameters to consider

event_date = pd.to_datetime('2025-02-10') # Super Bowl date is '2025-02-10' so if prior then this is a placebo test
preperiod_length = 90
postperiod_length = 3

# Define the start and end dates for the pre and post event periods: Use these to subset your data prior to estimating the models 
start_date = event_date - pd.DateOffset(days=preperiod_length)
end_date = event_date + pd.DateOffset(days=postperiod_length)

# Load the data


### 📌 **Plot the time series of each keyword**

### 📌 **Start with interrupted time series (ITS).** 

You only need the columns for date and nerdwallet search interest.  Consider that you make choices for the size of both the pre-treatment and post-treatment window. Plot the model predictions against actual values. How are the in-sample and out-of-sample fit? 

### 📌 **Next attempt Difference-in-Differences** 

It is not necessary for difference-in-differences to have just a single untreated comparison group. You will want to first melt the data so that there is one row per search term-x-date.  You should have columns for the date, search term, and the value. 

Make sure to test the critical assumptions for the DiD model. Again, you may want to consider the pre- and post-period windows and the in-sample and out-of-sample fit

### 📌 **Difference-in-Differences with controls** 

You can sometimes improve precision by including controls. One common approach is to include two-way fixed effects (TWFE). This includes fixed effects for both the date and search term. Make sure though you either exclude one of the categories as the baseline or remove the intercept.  Within smf.ols, you can remove the intercept by adding a '-1' inside the formula. 

How is the fit? Note if you model with date fixed effects, you will only be able to make in-sample predictions

### 📌 **Next attempt the Event Study** 

Unlike Difference-in-Differences, you do not want one row for each term-x-date in the Event Study. Instead, the data should be structured as it comes out of the Google Trends site with one column for each search term and one column for the dates.

First, set up the event study with a single 'post' variable. Then, estimate time-varying effects. Note that you can help validate the model during this step. How long do the effects appear to last?

### 📌 **Estimate a synthetic control using Lasso** 

As in the event study, you'll use data structured so that each search term is a separate column. Both the event study and synthetic control using Lasso estimate regression weights using pre-treatment data. Lasso makes it more likely that some terms will have a zero weight, whereas an exact zero weight is very unlikely in a standard regression model. This could be particularly important in real-world scenarios where you have many possible comparisons in the donor pool.

For consistency and simplicity in this first step, use Lasso(alpha=0.1) on pre-treatment data only to obtain the weights. Plot actual vs synthetic control values for the entire period through the end of the data. How is the fit? 

Inference using synthetic controls is a rapidly evolving field with recent academic papers even in 2025. One quick approach is to treat the synthetic control as your comparison time series and estimate a difference-in-differences model against it.  

### 📌 **Replicate the synthetic control but optimize model choices using cross validation** 

We previously used alpha=0.1 as a default. This time, use ML tools such as grid search and cross-validation to help tune and optimize alpha. Then re-estimate the synthetic control weights and DiD as before

### 📌 **Use permutation testing for inference for the Synthetic Control** 

Rather than using difference-in-differences to test how Nerdwallet search trends against the synthetic control, you will replicate the model looping over the donor pool, treating each as if they were treated.  Make sure that nerdwallet--the actual treated term--is NOT included in the donor pool as that will severely distort your results. 

For each of the potential donors, what is your SC estimate using the same framework as above? Take all those placebo estimates (in this case, there are just three), use the mean and standard deviation and calculate a 95% confidence interval. How does your actual estimate from Nerdwallet compare? 

### 📌 **Which is your preferred model?** 

Using data prior to the real event data, re-estimate the models and find the placebo effects. In a real-world situation, you would want to clarify with your stakeholders that there were no additional, new marketing events prior to the Super Bowl that could lead to positive 'placebo' effects. 

**Optional Extra Credit**: While you can get away with model selection using one pre-period date, you can enhance credibility by looping over dates and estimating a distribution of placebo effects. The mean and standard deviation of that distribution can be used for creating empirical confidence intervals. The RMSE of the placebo effects can be used for model selection.

### 📌 **Tell your story** 

Your stakeholders and concerned about the lack of randomized experimentation and are only vaguely familiar with causal inference methods. Make your pitch! What is the key takeaway? Which model did you choose and why? How confident are you in the results? ("Confidence" as used in everyday language, not statistical confidence. This common stakeholder question is often trying to get at: Are you sure the model isn't biased? Is it reasonably precise?)